In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as pl

In [2]:
fname='./L2/S1997263200238/S1997263200238_main.L2'

In [3]:
ds = nc.Dataset(fname)
ds.groups.keys()

odict_keys(['sensor_band_parameters', 'scan_line_attributes', 'geophysical_data', 'navigation_data', 'processing_control'])

In [10]:
gpv = ds.groups['geophysical_data'].variables

In [11]:
nav = ds.groups['navigation_data'].variables

In [20]:
latitude=nav['latitude'][:]

In [21]:
longitude=nav['longitude'][:]

In [22]:
latitude.shape,longitude.shape

((101, 101), (101, 101))

In [23]:
lat0,lon0=32.912,-117.393

In [24]:
a = abs( latitude - lat0 ) + abs( longitude - lon0 )
i,j = np.unravel_index(a.argmin(0),a.shape)

In [25]:
i,j

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([60, 60, 60, 59, 59, 59, 59, 59, 58, 58, 58, 58, 58, 58, 57, 57, 57,
        57, 57, 56, 56, 56, 56, 56, 55, 55, 55, 55, 55, 55, 54, 54, 54, 54,
        54, 54, 53, 53, 53, 53, 53, 52, 52, 52, 52, 52, 52, 51, 51, 51, 51,
        51, 51, 51, 50, 50, 50, 50, 50, 49, 49, 49, 49, 49, 48, 48, 48, 48,
        48, 48, 47, 47, 47, 47, 47, 47, 46, 46, 46, 46, 46, 45, 45, 45, 45,
        45, 45, 44, 44, 44, 44, 44, 43, 43, 43, 43, 43, 42, 42, 42, 42]))

In [27]:
print(latitude[i,j])

[ 34.87534714  34.87534714  34.87534714  34.88159943  34.88159943
  34.88159943  34.88159943  34.88159943  34.88902664  34.88902664
  34.88902664  34.88902664  34.88902664  34.88902664  34.89648819
  34.89648819  34.89648819  34.89648819  34.89648819  34.90441132
  34.90441132  34.90441132  34.90441132  34.90441132  34.9119873
  34.9119873   34.9119873   34.9119873   34.9119873   34.9119873
  34.91947174  34.91947174  34.91947174  34.91947174  34.91947174
  34.91947174  34.92692947  34.92692947  34.92692947  34.92692947
  34.92692947  34.93411636  34.93411636  34.93411636  34.93411636
  34.93411636  34.93411636  34.94119263  34.94119263  34.94119263
  34.94119263  34.94119263  34.94119263  34.94119263  34.94807053
  34.94807053  34.94807053  34.94807053  34.94807053  34.95465088
  34.95465088  34.95465088  34.95465088  34.95465088  34.96203995
  34.96203995  34.96203995  34.96203995  34.96203995  34.96203995
  34.96961594  34.96961594  34.96961594  34.96961594  34.96961594
  34.9696159

In [28]:
print(longitude[i,j])

[-116.2718277  -116.2718277  -116.2718277  -116.32428741 -116.32428741
 -116.32428741 -116.32428741 -116.32428741 -116.37644958 -116.37644958
 -116.37644958 -116.37644958 -116.37644958 -116.37644958 -116.42860413
 -116.42860413 -116.42860413 -116.42860413 -116.42860413 -116.48068237
 -116.48068237 -116.48068237 -116.48068237 -116.48068237 -116.53286743
 -116.53286743 -116.53286743 -116.53286743 -116.53286743 -116.53286743
 -116.58509827 -116.58509827 -116.58509827 -116.58509827 -116.58509827
 -116.58509827 -116.63736725 -116.63736725 -116.63736725 -116.63736725
 -116.63736725 -116.68968964 -116.68968964 -116.68968964 -116.68968964
 -116.68968964 -116.68968964 -116.74205017 -116.74205017 -116.74205017
 -116.74205017 -116.74205017 -116.74205017 -116.74205017 -116.79445648
 -116.79445648 -116.79445648 -116.79445648 -116.79445648 -116.84687805
 -116.84687805 -116.84687805 -116.84687805 -116.84687805 -116.89933777
 -116.89933777 -116.89933777 -116.89933777 -116.89933777 -116.89933777
 -116.

In [29]:
print(a[i,j])

[ 3.08451843  3.08451843  3.08451843  3.038311    3.038311    3.038311
  3.038311    3.038311    2.99357605  2.99357605  2.99357605  2.99357605
  2.99357605  2.99357605  2.94888306  2.94888306  2.94888306  2.94888306
  2.94888306  2.90472794  2.90472794  2.90472794  2.90472794  2.90472794
  2.86011887  2.86011887  2.86011887  2.86011887  2.86011887  2.86011887
  2.81537247  2.81537247  2.81537247  2.81537247  2.81537247  2.81537247
  2.77056122  2.77056122  2.77056122  2.77056122  2.77056122  2.72542572
  2.72542572  2.72542572  2.72542572  2.72542572  2.72542572  2.68014145
  2.68014145  2.68014145  2.68014145  2.68014145  2.68014145  2.68014145
  2.63461304  2.63461304  2.63461304  2.63461304  2.63461304  2.58877182
  2.58877182  2.58877182  2.58877182  2.58877182  2.54370117  2.54370117
  2.54370117  2.54370117  2.54370117  2.54370117  2.49874115  2.49874115
  2.49874115  2.49874115  2.49874115  2.49874115  2.45323944  2.45323944
  2.45323944  2.45323944  2.45323944  2.40749741  2.4

In [32]:
a.argmin(0)

array([60, 60, 60, 59, 59, 59, 59, 59, 58, 58, 58, 58, 58, 58, 57, 57, 57,
       57, 57, 56, 56, 56, 56, 56, 55, 55, 55, 55, 55, 55, 54, 54, 54, 54,
       54, 54, 53, 53, 53, 53, 53, 52, 52, 52, 52, 52, 52, 51, 51, 51, 51,
       51, 51, 51, 50, 50, 50, 50, 50, 49, 49, 49, 49, 49, 48, 48, 48, 48,
       48, 48, 47, 47, 47, 47, 47, 47, 46, 46, 46, 46, 46, 45, 45, 45, 45,
       45, 45, 44, 44, 44, 44, 44, 43, 43, 43, 43, 43, 42, 42, 42, 42])